# 🔰PyTorchでニューラルネットワーク基礎 #10 【時系列分析・指標計算】

## 内容
* Qiitaの記事と連動しています
* [9.5回指標の説明](https://qiita.com/AzukiImo/items/e02f9ec2422cddf01e46)で紹介した6種類を実際に計算するのが目的となります
* モデルの詳細については、[第9回](https://qiita.com/AzukiImo/items/e04318b6845aa609be8f)を参照


## データについて
* データ：data/nikkei_225.csv
* 2021年から2025年6月中旬くらいのデータ
* 日経225データはyfinanceを利用して取得

## ちょっとした注意点
* CPUでもできるけどGPU（ゲーム用ノートPC搭載のGPUでもOK）あると高速に学習できる
* Google ColaboratoryでももちろんOK


In [1]:
import pandas as pd
import numpy as np

In [2]:
# CSVファイルの読み込み
filename = "./data/nikkei_225.csv"
data = pd.read_csv(filename)
data.head(5)

Date      Open      High       Low     Close    Volume
0  2021-01-04  27575.57  27602.11  27042.32  27258.38  51500000
1  2021-01-05  27151.38  27279.78  27073.46  27158.63  55000000
2  2021-01-06  27102.85  27196.40  27002.18  27055.94  72700000
3  2021-01-07  27340.46  27624.73  27340.46  27490.13  98900000
4  2021-01-08  27720.14  28139.03  27667.75  28139.03  84900000

In [3]:
# 日経225の値を10,000円で割り算して値を小さくする
#
scaling_factor = 10_000
x_open = data["Open"]/scaling_factor
x_high = data["High"]/scaling_factor
x_low = data["Low"]/scaling_factor
x_close = data["Close"]/scaling_factor

In [4]:
win_size = 5

XO = [x_open[start:start+win_size] for start in range(len(data)-win_size)]
XH = [x_high[start:start+win_size] for start in range(len(data)-win_size)]
XL = [x_low[start:start+win_size] for start in range(len(data)-win_size)]
XC = [x_close[start:start+win_size] for start in range(len(data)-win_size)]

T = x_open[win_size:]

xo = np.array(XO)
xh = np.array(XH)
xl = np.array(XL)
xc = np.array(XC)
t  = np.array(T)
xo = xo.reshape(-1, win_size, 1)
xh = xh.reshape(-1, win_size, 1)
xl = xo.reshape(-1, win_size, 1)
xc = xh.reshape(-1, win_size, 1)

x = np.concatenate([xo, xh, xl, xc], axis=2)
x.shape, t.shape

((1087, 5, 4), (1087,))

## 3. LSTMを利用したネットワークの作成と学習

In [6]:
import torch
import torch.nn as nn

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")
x = torch.FloatTensor(x).to(device)
t = torch.FloatTensor(t).to(device).reshape(-1,1)


period = 100
x_train = x[:-period]
x_test = x[-period:]
t_train = t[:-period]
t_test = t[-period:]

x_train.shape, x_test.shape, t_train.shape, t_test.shape

device: cuda


(torch.Size([987, 5, 4]),
 torch.Size([100, 5, 4]),
 torch.Size([987, 1]),
 torch.Size([100, 1]))

### ネットワークの構造
4 dim --> LSTM --> FC1 --> FC2 --> 1 dim

In [8]:
## モデル定義
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm1  = nn.LSTM(input_size=4 ,hidden_size=100, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(100, 50)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        o, (h, c)  = self.lstm1(x)
        #o = o[:,-1,:]=h[-1] : 最後のステップの出力
        o = h[-1]
        h = self.fc1(o)
        h = self.act1(h)
        y = self.fc2(h)
        return y

In [9]:
model = DNN()
model.to("cuda")

DNN(
  (lstm1): LSTM(4, 100, batch_first=True)
  (fc1): Linear(in_features=100, out_features=50, bias=True)
  (act1): ReLU()
  (fc2): Linear(in_features=50, out_features=1, bias=True)
)

In [10]:
# ハイパーパラメータ
LOOP = 8_000 # 1000

In [11]:
# 損失関数と最適化関数の定義
criterion = nn.MSELoss() # 平均二乗誤差
optimizer = torch.optim.AdamW(model.parameters())

In [12]:
# 学習ループ
model.train()
for epoch in range(LOOP):
    optimizer.zero_grad()
    y = model(x_train)
    loss = criterion(y,t_train)           # 最小2乗誤差
    if (epoch+1)%1000 == 0:
        print(epoch,"\tloss:", loss.item())#, "\tR2:",r2.item())
    loss.backward()           # 逆伝播微分
    optimizer.step()          # パラメータ更新

999 	loss: 0.0020297644659876823
1999 	loss: 0.001471337745897472
2999 	loss: 0.001216319389641285
3999 	loss: 0.0010366884525865316
4999 	loss: 0.0009578355820849538
5999 	loss: 0.0009363829158246517
6999 	loss: 0.0009216913604177535
7999 	loss: 0.0009082588949240744


# テスト用データを使って検証
* xをtrainとtestで分割した。
* x_train = x[:-period]
* x_test = x[-period:]

* x_testとt_testを利用して、検証を行う。これは実際に学習されていないデータなので面白そう！

In [14]:
import matplotlib.pyplot as plt
import japanize_matplotlib

In [15]:
model.eval()
y_test = model(x_test)

prediction = y_test.detach().cpu().numpy()
real = t_test.detach().cpu().numpy()
y_train = t_train.detach().cpu().numpy()

## グラフの描画

In [16]:
s = 0 # s=0
e = s+50 # s+表示期間

plt.figure(figsize=(15,8))
plt.plot(real[s+1:e], label="real",marker="^")
plt.plot(prediction[s+1:e], label="prediction", linestyle="dotted", marker="*")
plt.plot(real[s:e-1], label="１期ずれ", marker=".")
plt.legend()
name = f"テストデータでの検証「最後の{s}期から{e}期まで」"
plt.title(name)
plt.show()

# いろんな指標でチェックしてみた
* ここからが本題

In [17]:
from sklearn.metrics import (
    mean_absolute_error,             # 平均絶対誤差
    root_mean_squared_error,         # 二乗平均平方誤差
    mean_absolute_percentage_error,  # 平均絶対パーセント誤差
    mean_squared_error,              # 平均二乗誤差
    r2_score                         # 決定係数
)

mae = mean_absolute_error(real, prediction)
rmse = root_mean_squared_error(real, prediction)
mape = mean_absolute_percentage_error(real, prediction)

# ついでにMSEとR2
mse = mean_squared_error(real, prediction)
r2 = r2_score(real, prediction)

print(f"MAE: {mae:.6f}")
print(f"RMSE: {rmse:.6f}")
print(f"MAPE: {mape*100:.6f}%")
print(f"MSE: {mse:.6f}")
print(f"R²: {r2:.6f}")

MAE: 0.025738
RMSE: 0.033230
MAPE: 0.702357%
MSE: 0.001104
R²: 0.964983


# 方向性精度

In [18]:
# Direction Accuracy
def direction_accuracy(y_true, y_pred):
    # 前日からの変化方向を計算
    true_direction = np.sign(np.diff(y_true.flatten()))
    pred_direction = np.sign(np.diff(y_pred.flatten()))  
    # 方向が一致した割合
    accuracy = np.mean(true_direction == pred_direction)
    return accuracy


dir_acc = direction_accuracy(real, prediction)
print(f"Direction Accuracy: {dir_acc:.3f} ({dir_acc*100:.1f}%)")

Direction Accuracy: 0.687 (68.7%)


# モデルの比較 MASEとDiebold Mariano検定
# MASE (mean absolute scaled error)

In [19]:
def mase(y_true, y_pred, y_train):
    # 予測のMAE
    mae_pred = np.mean(np.abs(y_true - y_pred))  
    # ナイーブ予測のMAE（前期の値を使用）
    mae_naive = np.mean(np.abs(y_train[1:] - y_train[:-1]))
    return mae_pred / mae_naive

mase_value = mase(real, prediction, y_train)
print(f"MASE: {mase_value:.3f}")

MASE: 0.945


# Diebold Mariano Test

In [20]:
from dieboldmariano import dm_test

actual = real[1:].reshape(-1)          # 実測値
f_model = prediction[1:].reshape(-1)   # モデルでの予測値
s_model = real[:-1].reshape(-1)        # ナイーブモデルでの予測値

# テストを実行
dm, p_value = dm_test(actual, f_model, s_model)

print(f"DM統計量: {dm:.5f}")
print(f"p値: {p_value:.5f}")
print(f"有意水準5%で差があるか: {'Yes' if p_value < 0.05 else 'No'}")

DM統計量: -2.59456
p値: 0.01092
有意水準5%で差があるか: Yes
